In [24]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.tools import Tool, tool
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor

load_dotenv()

True

In [15]:
from typing import Any
from uuid import UUID
from langchain_core.outputs import LLMResult

class AgentCallbackHandler(BaseCallbackHandler):
    def on_llm_start(
            self,
            serialized: dict[str, Any],
            prompts: list[str],
            *,
            run_id: UUID,
            parent_run_id: UUID | None = None,
            tags: list[str] | None = None,
            metadata: dict[str, Any] | None = None,
            **kwargs: Any,
        ) -> Any:
            """Run when LLM starts running.

            !!! warning
                This method is called for non-chat models (regular LLMs). If you're
                implementing a handler for a chat model, you should use
                `on_chat_model_start` instead.

            Args:
                serialized: The serialized LLM.
                prompts: The prompts.
                run_id: The run ID. This is the ID of the current run.
                parent_run_id: The parent run ID. This is the ID of the parent run.
                tags: The tags.
                metadata: The metadata.
                **kwargs: Additional keyword arguments.
            """
            print(f"Prompt to llm was: ***\n{prompts[0]}")
            print("******")
    
    def on_llm_end(
        self,
        response: LLMResult,
        *,
        run_id: UUID,
        parent_run_id: UUID | None = None,
        **kwargs: Any,
    ) -> Any:
        """Run when LLM ends running.

        Args:
            response: The response which was generated.
            run_id: The run ID. This is the ID of the current run.
            parent_run_id: The parent run ID. This is the ID of the parent run.
            **kwargs: Additional keyword arguments.
        """
        print(f"***LLM Response***:***\n{response.generations[0][0].text}")
        print("******")

In [18]:
@tool
def find_str_len(input: str) -> int:
    """Calculates the length of the given input by characters"""
    input = input.strip("\n").strip('"')
    
    return len(input)

In [ ]:
tools = [find_str_len]
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm = ChatOllama(model="gpt-oss:20b", temperature=0, callbacks=[AgentCallbackHandler()])
## New approach : bind tools from LLM Provider for better control on the tool calling rather than using regex in react.
llm_with_tools = llm.bind_tools(tools=tools)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpful assistant!"),
        ("user", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ]
)

agent = create_tool_calling_agent(llm=llm_with_tools, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(tools=tools, agent=agent)

In [27]:
if __name__ == '__main__':
    print("Ollama model Testing!")


    result = agent_executor.invoke(input={"input": "what is the length of 'DOG'?"})

    print(result)

Ollama model Testing!
Prompt to llm was: ***
System: you are a helpful assistant!
Human: what is the length of 'DOG'?
******
***LLM Response***:***

******
Prompt to llm was: ***
System: you are a helpful assistant!
Human: what is the length of 'DOG'?
AI: [{'name': 'find_str_len', 'args': {'input': 'DOG'}, 'id': 'b218d3f0-87c5-47cc-8bec-91164d345313', 'type': 'tool_call'}]
Tool: 3
******
***LLM Response***:***
The length of `'DOG'` is **3**.
******
{'input': "what is the length of 'DOG'?", 'output': "The length of `'DOG'` is **3**."}
